# RR-LL delay tests

[L. Blackburn, Sep 2018]

The purpose of this test is to check stability of RR-LL delay alignment for baselines in the array. We expect R-L delay to be stable at each antenna, thus baseline RR-LL delay should also be stable. RR-LL is a cleaner signal than the RL or LR necessary for a direct measurement of R-L at a single site because it has less relative contamination from leakage. Also because ALMA XY feeds are aligned, ALMA is able to be used as a reference by assuming R-L delay at ALMA is exactly zero.

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
from eat.hops import util as hu
from eat.plots import util as pu
from collections import OrderedDict
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.ticker import MultipleLocator
import seaborn as sns
sns.reset_orig()
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']
import sys
nb_stdout = sys.stdout # grab for later
def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()
def tightx(): plt.autoscale(enable=True, axis='x', tight=True)
def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')
def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])
# pd.options.display.float_format = '{:,.6f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))
# Run this in Python once, it should take effect permanently
from notebook.services.config import ConfigManager
c = ConfigManager()
_ = c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})
import string

In [ ]:
os.getcwd()

In [ ]:
reload(util)
# define and load data
alistf = 'alist.v6'
stage = '3.+adhoc'
# datadir = '/data/2017-april/ce/er3/hops-lo/%s/data/' % stage
datadir = os.environ['DATADIR'] + '/'
a = util.noauto(hops.read_alist(datadir + alistf))
util.fix(a)
util.unwrap_mbd(a)
util.add_days(a)
util.add_delayerr(a)
util.add_path(a)
util.add_scanno(a)

In [ ]:
# data filters
a = a[(a.snr > 7.) & ~a.baseline.isin({'RS', 'SR'})].copy()

In [ ]:
# expt_no boundary ticker marks vs scan
elines = a.scan_no.sort_values().values[np.nonzero(np.diff(a.expt_no.sort_values()) > 0)[0]] + 0.5

In [ ]:
# calculate the segmented statistics
(p, stats) = hu.rrll_segmented(a, restarts=hu.restarts_2017)
wa = [bl for bl in set(p.index.get_level_values('baseline')) if bl[0] == 'A']
na = [bl for bl in set(p.index.get_level_values('baseline')) if bl[0] != 'A']

In [ ]:
stats.loc[(slice(None),slice(None),wa),:]

In [ ]:
plt.subplot(3, 1, 1)
hu.rrllplot(p, baselines=wa, vlines=elines)
plt.xlim(0, 1.05*plt.xlim()[1])
plt.title('RR-LL delay after subtracting mean value [%.0f MHz]' % (p.iloc[0].ref_freq))
plt.subplot(3, 1, 2)
hu.rrllplot(p, baselines=na[:len(na)//2], vlines=elines)
plt.xlim(0, 1.05*plt.xlim()[1])
plt.subplot(3, 1, 3)
hu.rrllplot(p, baselines=na[len(na)//2:], vlines=elines)
plt.xlim(0, 1.05*plt.xlim()[1])
wide(12, 10)

In [ ]:
# table of outliers
outliers = (p.LLRR_offset.abs() > 0.000050) & (p.LLRR_std.abs() > 5)
p.loc[outliers, "expt_no scan_id source timetag baseline mbd_unwrap LLRR_offset LLRR_std".split()]

In [ ]:
# scatter plot shows balancing between systematic error and bandwidth inflation factor
# we want to see relatively well-behaved distribution across multiple SNR
plt.semilogx(p['LLRR_err'].values, p['LLRR_std'].values, '.')
plt.ylim(-5, 5)
plt.xlabel('predicted LL-RR error')
plt.ylabel('sigmas away from mean')
# plt.axhline(0, ls='--', color='k', alpha=0.25)
plt.gca().yaxis.grid(ls='--', alpha=0.5)
wide(10, 5)

In [ ]:
# histogram of sigmas deviation
from scipy.stats import norm
lim = 8 # np.ceil(np.max(np.abs(p.LLRR_std)))
xx = np.linspace(-lim, lim, 200)
bins = np.linspace(-lim, lim, 161)
for baselines in [wa, na]:
    q = p.loc[(slice(None),slice(None),baselines),:]
    (names, vals) = zip(*[(bl, rows.LLRR_std) for (bl, rows) in q.groupby('baseline')])
    names2 = (name + ': %.1f' % np.sqrt(np.mean(val**2))
              for (name, val) in zip(names, vals))
    plt.hist(vals, bins=bins,
        histtype='barstacked', alpha=1.0, label=names2, normed=1.0)
    std = np.mean(q.LLRR_std**2)
    plt.plot(xx, norm.pdf(xx, loc=0, scale=1.0), 'k--', alpha=0.5)
    plt.xlabel('std away from mean')
    plt.ylabel('distribution of scans')
    plt.title('RR-LL delay offsets after subtracting mean value [%.0f MHz]' % (p.iloc[0].ref_freq))
    plt.legend(loc='upper right')
    plt.grid(alpha=0.25)
    pu.tag('N = %d, std=%.1f' % (len(q), std), loc='upper left')
    plt.xlim(-lim, lim) # only show bulk distribution
    wide(12, 4.5)
    plt.show()